In [1]:
#!nvidia-smi

In [2]:
import sys
sys.path.append('keras_elmo_bert/')

import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import itertools
import string
import numpy as np 
import pandas as pd 
import tensorflow as tf
import tensorflow_hub as hub
import re

from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.layers import Layer
from keras.utils import Sequence
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, Flatten, GlobalMaxPool1D, LSTM, Dot, Lambda
from keras.layers import Input, MaxPooling1D, GlobalAveragePooling1D, multiply, concatenate, Reshape
from keras.layers import Bidirectional
from keras.optimizers import Adam
import keras.initializers as initializers
import keras.regularizers as regularizers
import keras.constraints as constraints


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle
import pickle

sess = tf.Session()

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

W0811 21:40:15.668303 139698212833088 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [4]:
max_seq_length = 64
bert_path = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"
elmo_path = "https://tfhub.dev/google/elmo/2"

# Create train test split

Чтобы понять обобщающую способность модели, необходимо валидироватсья не тех обьектах, которые не участвовали в обучении. Соотвественно, мы выбираем некоторое количество индексов и смотрим, чтобы они не находились в валидации в любом из полей qid1 или qid2  

In [6]:
all_ques_df = pd.read_pickle('data/all_ques_df.pickle')
df = pd.read_pickle('data/df.pickle')

In [7]:
qid_arr = np.unique(all_ques_df['new_qid'].values)

In [8]:
train_ind, val_ind =  train_test_split(qid_arr, test_size=0.3, random_state=42)

In [9]:
train_df = df[((df.qid1_new.isin(train_ind)) | (df.qid2_new.isin(train_ind)))]

In [10]:
test_df = df[((~df.qid1_new.isin(train_ind)) & (~df.qid2_new.isin(train_ind)))]

In [11]:
print(f'Train size is {len(train_df)/len(df)}')
print(f'Test size is {len(test_df)/len(df)}')

Train size is 0.9112964518580743
Test size is 0.08870354814192567


# ELMO pipeline

## Elmo Tokenizer

In [41]:
from elmo_tokenizer import ELMO_tokenizer

In [42]:
tokenizer = ELMO_tokenizer(max_seq_length)

In [15]:
train_text = [tokenizer.predict(train_df['question1'].tolist()), tokenizer.predict(train_df['question2'].tolist())]
train_label = train_df['is_duplicate'].values

test_text = [tokenizer.predict(test_df['question1'].tolist()), tokenizer.predict(test_df['question2'].tolist())]
test_label = test_df['is_duplicate'].values

## Elmo Model

In [26]:
from elmo_layer import ElmoLayer

In [27]:
def build_model_elmo_siamese(max_seq_length): 
    
    input_tokens_1 = Input(shape=(max_seq_length,),dtype="string")
    input_tokens_2 = Input(shape=(max_seq_length,),dtype="string")
    
    def get_elmo_with_head():
        input_tokens = Input(shape=(max_seq_length,),dtype="string")
        elmo_output = ElmoLayer(trainable=True, tf_hub = elmo_path, output_representation='default')(input_tokens)
        dense = Dense(256, activation='sigmoid')(elmo_output)
        
        return  Model(inputs=input_tokens, outputs=dense)
    
    ELMO = get_elmo_with_head()
    dense_tokens_1 = ELMO(input_tokens_1)
    dense_tokens_2 = ELMO(input_tokens_2)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([dense_tokens_1, dense_tokens_2])
    
    pred = Dense(1, activation='sigmoid')(L1_distance)
    model = Model(inputs=[input_tokens_1,input_tokens_2 ], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [43]:
elmo_model = build_model_elmo_siamese(max_seq_length)
initialize_vars(sess)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
model_13 (Model)                (None, 256)          93863252    input_22[0][0]                   
                                                                 input_23[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256)          0           model_13[1][0]                   
          

In [44]:
elmo_model.trainable_weights

[<tf.Variable 'elmo_layer_8_module/aggregation/weights:0' shape=(3,) dtype=float32>,
 <tf.Variable 'elmo_layer_8_module/aggregation/scaling:0' shape=() dtype=float32>,
 <tf.Variable 'dense_9/kernel:0' shape=(1024, 256) dtype=float32_ref>,
 <tf.Variable 'dense_9/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_10/kernel:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32_ref>]

In [ ]:
elmo_model.fit(
    train_text,
    train_label,
    validation_data=(test_text, test_label),
    epochs=1,
    verbose = 1,
    batch_size=32
)


Train on 91126 samples, validate on 8870 samples
Epoch 1/1
 8672/91126 [=>............................] - ETA: 14:47 - loss: 0.6030 - acc: 0.6369

## Load ELMO model

In [29]:
elmo_model = build_model_elmo_siamese(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
model_3 (Model)                 (None, 256)          93863252    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256)          0           model_3[1][0]                    
          

In [30]:
elmo_model.load_weights('models/ELMOModel.h5')

## Find 100 most similar objects for qid =1

In [31]:
all_ques_df.head()

,qid,questions,num_of_words,new_qid
0,1,What is the step by step guide to invest in sh...,14,1
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...,8,3
2,5,How can I increase the speed of my internet co...,14,5
3,7,Why am I mentally very lonely? How can I solve...,11,7
4,9,"Which one dissolve in water quikly sugar, salt...",13,9


In [32]:
search_df = all_ques_df.query('new_qid < 121424')

In [33]:
search_df = search_df.drop_duplicates('new_qid')

In [34]:
qid_1_question = search_df.query('qid == 1').questions.values[0]

In [35]:
search_df = search_df[['questions','qid']]

In [36]:
search_df.head()

,questions,qid
0,What is the step by step guide to invest in sh...,1
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,3
2,How can I increase the speed of my internet co...,5
3,Why am I mentally very lonely? How can I solve...,7
4,"Which one dissolve in water quikly sugar, salt...",9


In [37]:
search_df['qid_1_question'] = qid_1_question

In [38]:
search_df.head()

,questions,qid,qid_1_question
0,What is the step by step guide to invest in sh...,1,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,3,What is the step by step guide to invest in sh...
2,How can I increase the speed of my internet co...,5,What is the step by step guide to invest in sh...
3,Why am I mentally very lonely? How can I solve...,7,What is the step by step guide to invest in sh...
4,"Which one dissolve in water quikly sugar, salt...",9,What is the step by step guide to invest in sh...


In [43]:
search_text = [tokenizer.predict(search_df['qid_1_question'].tolist()), tokenizer.predict(search_df['questions'].tolist())]

In [45]:
prediction = elmo_model.predict(search_text, verbose=1, batch_size=128)

106103/106103 [==============================] - 788s 7ms/step


In [46]:
top100_ind = prediction.argsort(axis=0)[::-1][:100,0]

In [56]:
pd.set_option('max_colwidth', 200)
search_df.iloc[top100_ind].to_csv('results/top100_prediction_elmo.csv')

In [54]:
search_df['prediction'] = prediction.flatten()

In [58]:
search_df.to_csv('results/all_prediction_elmo.csv')